In [1]:
    import cvxpy as cp
import numpy as np

def name_of_inequality(inequality):
    if inequality == leq:
        return "<"
    elif inequality == geq:
        return ">"
    else:
        raise ValueError("Unknown inequality")

def print_constraints(inequalities, last_inequality, n, k):
    print(f"[gamma_{0, 0}", end="")
    for i in range(k): # for each repetition
        for j in range(n - 1): # for each segment in the sequence
            print(f" {name_of_inequality(inequalities[j])} gamma_{i, j + 1}", end="")
        print("]", end=" ")
        if i < k - 1:
            print(f"{name_of_inequality(last_inequality)} [gamma_{i + 1, 0}", end=" ")

constraints = []

leq = lambda x, y: x <= y
geq = lambda x, y: x >= y

n = 3 # number of segments in the segment sequence
inequalities = [leq, geq]; assert len(inequalities) == n - 1
last_inequality = geq
c = np.array([-50, 1, 100]) # cost vector
b = 200 # additional bias term

k = 1 # number of times we want to repeat the segment sequence
gammas = cp.Variable((k, n)) # gammas[i, j] is the gamma for the jth segment in the ith repetition

for i in range(k):
    for j, inequality in enumerate(inequalities):   # intra-sequence connecting constraints
        constraints.append(inequality(gammas[i, j], gammas[i, j + 1]))

for i in range(k - 1): # inter-sequence connecting constraints
    constraints.append(last_inequality(gammas[i, n - 1], gammas[i + 1, 0]))

constraints.append(gammas >= -1)
constraints.append(gammas <= 1)
constraints.append(gammas==0)

objective = cp.Minimize(cp.sum(gammas @ c + np.ones(k) * b))

prob = cp.Problem(objective, constraints)
prob.solve()

print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var\n", gammas.value)

print_constraints(inequalities, last_inequality, n, k)

status: optimal
optimal value 199.99999999999343
optimal var
 [[-8.52821460e-13  1.53428315e-12 -5.07526895e-13]]
[gamma_(0, 0) < gamma_(0, 1) > gamma_(0, 2)] 